In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from numpy import zeros, ones
from numpy.random import randn, randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
import numpy as np
import cv2
from PIL import Image
import random
import re
from keras.preprocessing.image import img_to_array

from tqdm import tqdm

In [ ]:
#Define input image dimensions
img_rows = 128
img_cols = 128
channels = 1
img_shape = (img_rows, img_cols, channels)

In [ ]:
def define_discriminator(in_shape=(128,128,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(128, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 64x64
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 32x32
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 16x16
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 8x8
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
#Verify the model summary
test_discr = define_discriminator()
print(test_discr.summary())
plot_model(test_discr, to_file='disc_model.png', show_shapes=True)

In [ ]:
# define the standalone generator model
# Generator must generate 128x128x3 images that can be fed into the discriminator. 
# So, we start with enough nodes in the dense layer that can be gradually upscaled
#to 128x128x3. 
def define_generator(latent_dim):
    model = Sequential()
    # Define number of nodes that can be gradually reshaped and upscaled to 128x128x3
    n_nodes = 128 * 8 * 8 #8192 nodes
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8, 8, 128)))
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 128x128
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer 128x128x3
    model.add(Conv2D(3, (8,8), activation='tanh', padding='same')) #tanh goes from [-1,1]
    return model

In [ ]:
test_gen = define_generator(100)
print(test_gen.summary())
plot_model(test_gen, to_file='generator_model.png', show_shapes=True)

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
test_gan = define_gan(test_gen, test_discr)
print(test_gan.summary())
plot_model(test_gan, to_file='combined_model.png', show_shapes=True)

In [ ]:
# Function to sample some random real images
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1)) # Class labels for real images are 1
    return X, y

In [ ]:
# Function to generate random latent points
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim) #Reshape to be provided as input to the generator. 
    return x_input

# Function to generate fake images using latent vectors
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples) #Generate latent points as input to the generator
    X = g_model.predict(x_input) #Use the generator to generate fake images
    y = zeros((n_samples, 1)) # Class labels for fake images are 0
    return X, y

In [ ]:
# Function to save Plots after every n number of epochs
def save_plot(examples, epoch, n=3):
    # scale images from [-1,1] to [0,1] so we can plot
    examples = (examples + 1) / 2.0
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i])
    # save plot to a file so we can view how generated images evolved over epochs
    filename = 'generated_plot_128x128_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

In [ ]:
# Function to summarize performance periodically. 
# 
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # Fetch real images
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real images - get accuracy
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # Generate fake images
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake images - get accuracy
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # Print discriminate accuracies on ral and fake images. 
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save generated images periodically using the save_plot function
    save_plot(x_fake, epoch)


In [ ]:
# train the generator and discriminator by enumerating batches and epochs. 

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2) #Disc. trained on half batch real and half batch fake images
    #  enumerate epochs
    for i in range(n_epochs):
        # enumerate batches 
        for j in range(bat_per_epo):
            # Fetch random 'real' images
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # Train the discriminator using real images
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' images 
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # Train the discriminator using fake images
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # Generate latent vectors as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # Label generated (fake) mages as 1 to fool the discriminator 
            y_gan = ones((n_batch, 1))
            # Train the generator (via the discriminator's error)
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # Report disc. and gen losses. 
            print('Epoch>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
            # save the generator model
            filename = 'generator_model_128x128_%03d.h5' % (i+1)
            g_model.save(filename)


In [ ]:
# to get the files in proper order
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

In [ ]:
SIZE = 128
_img = []
path = '../input/face-mask-lite-dataset/without_mask'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):     
    img = cv2.imread(path + '/'+i,1)
    # open cv reads images in BGR format so we have to convert it to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #resizing image
    img = cv2.resize(img, (SIZE, SIZE))
    img = (img - 127.5) / 127.5
    img = img.astype(float)
    _img.append(img_to_array(img))

In [ ]:
dataset = np.array(_img)
# size of the latent space
latent_dim = 100
# create the discriminator using our pre-defined function
d_model = define_discriminator()
# create the generator using our pre-defined function
g_model = define_generator(latent_dim)
# create the gan  using our pre-defined function
gan_model = define_gan(g_model, d_model)

# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=500)